In [ ]:
# Load test data

import json

with open("test.json", "r") as json_file:
    test_data = json.load(json_file)

In [ ]:
def format_input(entry):
    instruction_text = (
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

print(format_input(test_data[0]))

In [ ]:
input_thesis = {
    "instruction" : """A Multi-Task Neural Network (MTNN) is a type of network in the field of machine learning
 where a single large neural network learns to solve multiple tasks simultaneously. In this paper,
 the performance of MTNNs is compared with that of corresponding Single-Task Neural Networks
 (STNN). The primary focus of this paper is on four autonomous driving perception tasks: semantic
 segmentation, lane marking, drivable area, and object detection. The Audi Autonomous Driving
 Dataset (A2D2) is used for the experiments. The dataset contains semantic segmentation labels
 for 57 classes (including labels for lane marking and drivable area) and 2D bounding boxes for
 14 classes for 12,497 images. Here, we first implement and train STNNs, then MTNNs for all
 the combinations of our four primary tasks. The MTNN used here has a shared encoder and one
 decoder for each task. Introduced new loss weighing and training techniques for better optimization
 of MTNNs. Based on the experiment results, multi-task learning outperforms single-task learning
 both in speed and accuracy. MTNNs are up to 33% faster, with improvement in accuracy of
 semantic segmentation up to 1%, lane marking up to 2.5%, drivable area up to 1.5%, and object
 detection up to 12%. Further, transfer learning method is used to train STNNs with MTNN’s
 shared encoder weights. Results show improvement in STNNs accuracy up to 2%. Also, it shows
 that MTNN’s shared encoder has learned better generalized feature representations on the A2D2
 dataset. Finally, an MTNN with object detection as its primary task and semantic segmentation
 as an auxiliary task is trained on A2D2 dataset. The labels for auxiliary task were generated using
 DeepLabv3 architecture model pre-trained on CityScapes and BDD dataset. The results show that
 the object detection mAP score was improved by 6.94%.""",
    "input" : "What is MTNN?"
}

In [ ]:
from litgpt import LLM

llm = LLM.load("meta_llama_32_finetuned_on_sqaud/finetune/lora/final")

In [ ]:
llm2 = LLM.load("tiny_llama_lora_fintune_on_squad_v1/final")

In [ ]:
llm3 = LLM.load("tiny_llama_lora_fintune_on_squad_v1-v2-fu rther2-lr6e1/final")
llm4 = LLM.load("tiny_llama_lora_fintune_on_squad_v1-v2-further-lr6e6/final")
llm5 = LLM.load("tiny_llama_lora_finetune_on_squad_v1-v2/final")


In [ ]:
import json
from transformers import AutoTokenizer
from tqdm import tqdm

# Load the training data
with open("qa_dataset/train_squad_v1_v2_hotpot.json", "r") as json_file:
    train_data = json.load(json_file)

# Initialize the tokenizer (use a general tokenizer, like LLama or GPT-2)
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Replace with your model's tokenizer if different

# Function to format input as in your provided code
def format_input(entry):
    instruction_text = f"\n\n### Instruction:\n{entry['instruction']}"
    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    return instruction_text + input_text

# Calculate the maximum token size
max_token_count = 0
max_entry = None  # To keep track of the longest entry
tokens_morethan_2048 = 0

examples_less_than_2048 = []
for entry in tqdm(train_data):
    formatted_text = format_input(entry)
    tokenized = tokenizer(formatted_text, truncation=False, return_tensors=None)
    token_count = len(tokenized["input_ids"])
    
    if token_count > 1800:
        tokens_morethan_2048 += 1
        continue
    examples_less_than_2048.append(entry)
        #max_entry = entry
print(f"Examples more than 2048 tokens : {tokens_morethan_2048}")
#print(f"Maximum token size: {max_token_count}")
#print("Longest entry:")
#print(format_input(max_entry))

with open("train_v1_v2_hotpot_max_1800.json", "w") as file2:
    json.dump(examples_less_than_2048, file2, indent=4)

"""
for test_squad_v1_v2_hotpot.json
Examples more than 2048 tokens : 112
Maximum token size: 0
Longest entry:

for train_squad_v1_v2_hotpot.json
Examples more than 2048 tokens : 2175
Examples more than 2000 tokens : 

"""

In [ ]:
test_sample = input_thesis
test_sample["input"] = "How much MTNNs are better than STNNs in terms of inference speed?"
print("llm1",llm.generate(format_input(test_sample), top_k=1))
print("llm2",llm2.generate(format_input(test_sample), top_k=1))
print("llm3", llm3.generate(format_input(test_sample), top_k=1))
print("llm4", llm3.generate(format_input(test_sample), top_k=1))
print("llm5", llm3.generate(format_input(test_sample), top_k=1))

In [ ]:
import json
from transformers import AutoTokenizer
from tqdm import tqdm

# Load the training data
with open("qa_dataset/train_squad_v1_v2_hotpot.json", "r") as json_file:
    train_data = json.load(json_file)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Replace with your model's tokenizer if different
max_token_length = 1400  # Define max token limit

# Function to format input
def format_input(entry):
    instruction_text = f"\n\n### Instruction:\n{entry['instruction']}"
    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    return instruction_text + input_text

# Filter entries
examples_less_than_max_tokens = []
tokens_more_than_limit = 0

for idx, entry in enumerate(tqdm(train_data, desc="Processing entries")):
    try:
        formatted_text = format_input(entry)
        tokenized = tokenizer(formatted_text, truncation=False, return_tensors=None)
        token_count = len(tokenized["input_ids"])

        # Debug print for token count
        if idx < 5:  # Print for the first few entries only
            print(f"Entry {idx} token count: {token_count}")
            print(f"Formatted text: {formatted_text[:100]}...")  # Print a snippet of the text

        if token_count > max_token_length:
            tokens_more_than_limit += 1
            continue

        examples_less_than_max_tokens.append(entry)
    except KeyError as e:
        print(f"Missing key in entry {idx}: {e}")

# Summary
print(f"Total examples: {len(train_data)}")
print(f"Examples exceeding {max_token_length} tokens: {tokens_more_than_limit}")
print(f"Examples within token limit: {len(examples_less_than_max_tokens)}")

# Save filtered data
output_file = "train_v1_v2_hotpot_1400.json"
with open(output_file, "w") as file:
    json.dump(examples_less_than_max_tokens, file, indent=4)
print(f"Filtered data saved to {output_file}")
